  \begin{equation}\label{eq:dsg}
    \ddot{x}_{\ell}=k(x_{\ell+1}-2x_{\ell}+x_{\ell-1})-g\sin{x_{\ell}}
  \end{equation}
  $x_7=\sin(x_1), x_8=\sin(x_3), x_9=\sin(x_5), x_{10}=\cos(x_1), x_{11}=\cos(x_3), x_{12}=\cos(x_5)$

In [3]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'C:\Users\mebratie\Desktop\KR\KR_Regression\Data\discrete_Sine-Gordon\DSG_T8_N100_p'
def generate_random_values():
    min_val = 0.1
    max_val = 2
    x1 = random.uniform(min_val, max_val)
    x2 = random.uniform(min_val, max_val)
    x3 = random.uniform(min_val, max_val)
    x4 = random.uniform(min_val, max_val)
    x5 = random.uniform(min_val, max_val)
    x6 = random.uniform(min_val, max_val)
    x7 = random.uniform(0.5, 1.2)
    x8 = random.uniform(0.5, 1.2)
    return x1, x2, x3, x4, x5, x6, x7, x8
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3, x4, x5, x6, x7, x8 = y
        f = np.array([x2, x7*(x3 - 2*x1 + x5) - x8*math.sin(x1), x4, x7*(x5 - 2*x3 + x1) - x8*math.sin(x3), x6, x7*(x1 - 2*x5 + x3) - x8*math.sin(x5), 0, 0])
        normalized_f = normalize(f)
        return normalized_f
    num_trajectories = 8
    t = np.linspace(0, 50, 100) # 10 data points per trajectory
    all_trajectory_data = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3, x4, x5, x6, x7, x8):")
    for i, initial_condition in enumerate(initial_conditions):
        print(f"({initial_condition[0]}, {initial_condition[1]}, {initial_condition[2]}, {initial_condition[3]}, {initial_condition[4]}, {initial_condition[5]}, {initial_condition[6]}, {initial_condition[7]})")
        sol = odeint(normalized_system, initial_condition, t)
        all_trajectory_data.append(sol)
    num_variables = 14 # Adjust number of variables that we need for the regression accordingly
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')
    with open('50.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, data in enumerate(all_trajectory_data):
            for j in range(len(t)):
                x1 = data[j, 0]
                x2 = data[j, 1]
                x3 = data[j, 2]
                x4 = data[j, 3]
                x5 = data[j, 4]
                x6 = data[j, 5]
                x7 = data[j, 6]
                x8 = data[j, 7]
                writer.writerow([x1, x2, x3, x4, x5, x6, x7, x8, math.sin(x1), math.sin(x3), math.sin(x5), math.cos(x1), math.cos(x3), math.cos(x5), r+1]) 
    output_directory1 = r'C:\Users\mebratie\Desktop\KR\KR_Regression\results\discrete_Sine-Gordon\DSG_T8_N100_p'
    plt.figure(figsize=(10, 6))
    for i, sol in enumerate(all_trajectory_data):
        for j in range(sol.shape[1]):
            plt.plot(t, sol[:, j])
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open('C:\\Users\\mebratie\\Desktop\\KR\\KR_Regression\\Data\\discrete_Sine-Gordon\\DSG_T8_N100_p\\trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    x1, x2, x3, x4, x5, x6, x7, x8 = generate_random_values()
    initial_conditions = [generate_random_values() for _ in range(8)]
    generate_data(initial_conditions)
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 9):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3, x4, x5, x6, x7, x8):
(0.5917079580855759, 1.5012937230183656, 1.8064778803842312, 1.0749249387947695, 1.4331781616840165, 1.331890699002767, 0.5619495616409014, 1.0938154720040076)
(0.4973290577484597, 1.9153366137962304, 0.6725324529606967, 1.1516028934448133, 1.365989366270432, 1.2206329583313367, 0.8840319983451419, 0.7401697457030413)
(0.21448166686286751, 0.8080560921181196, 1.8057860989715777, 1.7969599949580015, 0.10318031970182238, 0.21188557902879007, 0.5030671426570873, 1.0215949697713191)
(1.9301480263522874, 1.027480520958983, 1.1975157282691131, 1.2065041983965437, 1.931631819679248, 0.7780223563655903, 0.6078461517407288, 0.8939725329232231)
(1.7189193965745924, 1.2427464238395125, 1.772995562984308, 1.3563491614911694, 0.5913770763171673, 1.34031052743594, 0.9932230099599448, 1.1984042646986466)
(1.202564623469708, 0.4780622034828974, 0.5593896595469339, 0.40924120172708545, 1.6781015763049614, 1.1996838985087632, 0.5611131347211515, 0.649071360